In [24]:
import pymongo
import collections
import numpy as np
import pandas as pd

In [2]:
db = pymongo.MongoClient()

In [10]:
def get_author(doc_id):
    try:
        return list(db["datasets"]["habrahabr"].find({"_id": doc_id}, {"authors_names": 1}))[0]["authors_names"][0]
    except Exception:
        return None

In [15]:
clf_output = pd.read_csv("classifier_output.csv")
clf_output.columns = ["id", "proba"]
clf_output = clf_output.set_index("id")["proba"]

In [21]:
%%time

authors_relevance = collections.defaultdict(list)

for doc_id, p in clf_output.items():
    author_name = get_author(doc_id)
    authors_relevance[author_name].append(p)

In [112]:
P = len(clf_output[clf_output >  0.5]) / len(clf_output)
N = len(clf_output[clf_output <= 0.5]) / len(clf_output)

def relevance(n, p=0):
    return np.sqrt(p / P) - np.sqrt(n / N)

def cont_relevance(ps):
    return np.log10(len(ps)) * np.median(2 * ps - 1)

In [113]:
authors_series = pd.Series({k: relevance(*np.bincount(np.array(v) > 0.5)) if v else 0
                            for k, v in authors_relevance.items()})

In [97]:
authors_series = pd.Series({k: cont_relevance(np.array(v)) if v else 0
                            for k, v in authors_relevance.items()})

In [126]:
with open("popular-sciency-rating.txt", "w") as f:
    for i, (author_name, r) in enumerate(authors_series.sort_values(ascending=False).items()):
        s = "% 10.6f -- https://habrahabr.ru/users/%s\n" % (r, author_name)
        if i < 10 or i >= len(authors_series) - 10:
            print(s, end="")
        elif i == 10:
            print("  ...")
        f.write(s)

 22.938461 -- https://habrahabr.ru/users/SLY_G
 18.314086 -- https://habrahabr.ru/users/lozga
 14.878677 -- https://habrahabr.ru/users/Zelenyikot
 12.548149 -- https://habrahabr.ru/users/krasandm
 12.498694 -- https://habrahabr.ru/users/Synth
 11.519339 -- https://habrahabr.ru/users/protogui
 11.377467 -- https://habrahabr.ru/users/AlexeyR
 11.285316 -- https://habrahabr.ru/users/LukinB
 10.340973 -- https://habrahabr.ru/users/PatientZero
  9.634685 -- https://habrahabr.ru/users/Boomburum
  ...
 -7.757439 -- https://habrahabr.ru/users/BBSoD
 -7.815363 -- https://habrahabr.ru/users/Tylerskald
 -8.655005 -- https://habrahabr.ru/users/azproduction
 -8.725304 -- https://habrahabr.ru/users/XaocCPS
 -9.768331 -- https://habrahabr.ru/users/azazelis
-11.758759 -- https://habrahabr.ru/users/aleksandrit
-11.796221 -- https://habrahabr.ru/users/RoboForm
-13.036321 -- https://habrahabr.ru/users/marks
-15.156126 -- https://habrahabr.ru/users/alizar
-16.058914 -- https://habrahabr.ru/users/jeston


---